<center>
<h1>STAT 654: Statistical Computing with R and Python</h1>
<h2>Bayesian Computations: Gibbs Samplers</h2>
<strong>
Daniel Drennan<br>
Professor Sharmistha Guha<br><br>
Department of Statistics<br>
Texas A&M University<br>
College Station, TX, USA<br><br>
Spring 2022<br>
</strong>
</center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

## Introduction

This notebook covers the implementation of Gibbs samplers to sample from the posterior distribution in a Bayesian statistical
model. It assumes some background on Bayesian computations from the class lectures and course materials, but recaps some of 
that material here for completeness. After the review, an example of sampling from a t distribution illustrates how to set up
a Bayesian statistical model, how to evaluate sampler performance, and how to summarize inference using the posterior
distribution.

### Motivation

Suppose $y_1, \ldots, y_n | \theta \overset{iid}{\sim} f(\cdot | \theta)$ for density (or mass) function $f$
parameterized by a vector $\theta$. In classical statistics, we consider $\theta$ to be a fixed but unknown quantity to be
estimated from a random sample using a likelihood function $L(\theta) = \prod_{i=1}^{n} f(y_i | \theta)$. This is known as
likelihood-based inference, where our estimates for $\theta$ are the maximum likelihood estimators $\hat{\theta}_{\text{MLE}}$.

In a Bayesian analysis, we assume $\theta$ is a random variable following a prior distribution $\pi$. Using Bayes' rule, 
we can compute a posterior distribution as
\begin{equation}
    \pi(\theta | y_1, \ldots, y_n) = \frac{
        \prod_{i=1}^{n}f(y_i | \theta) \pi(\theta)
        }{
            \int_{0}^{\infty}\prod_{i=1}^{n}f(y_i | \theta) \pi(\theta) d\theta.
        }
        \tag{1}
\end{equation}
The posterior distribution combines all of the information contained in the prior $\pi$ and the likelihood $L$, and is
offers a principled way to update our uncertainty about $\theta$. Once a posterior distribution is obtained, it can be
used to generate credible sets and summary statistics from the distribution, such as estimates of the mean and variance
of the distribution. To perform a Bayesian analysis, we must be able to calculate the integral in $(1)$ either analytically
or numerically. For a small class of models, called conjugate models, this is trivially easy to do. However, the integral
is not generally easy to compute for complex models. In the non-conjugate models, summarizing $\pi(\theta | y)$ requires
drawing random samples from the posterior distribution and then computing parameters such as the mean, variance, and skew
using those samples.

### Markov Chain Monte Carlo

Markov chain Monte carlo (MCMC) is a rich theory which is applicable to this sampling problem, and the Gibbs sampling algorithm
is method for sampling from posterior distributions. The Gibbs sampler is useful in problems where we can write hierarchical
models on the parameter space. For a joint probability distribution $\pi(u, v)$, two hierarchical models are 
$\pi(u, v) = \pi(u | v) \pi(v) = \pi(v | u) \pi(u)$ (notice these are all equal!). Our goal is to draw samples from $\pi(u, v)$.
If we can write analytic expressions for $\pi(u | v)$ and $\pi(v | u)$, we can use a Gibbs sampler to draw approximate
samples from $\pi(u, v)$. 

The general approach to running a Gibbs sampler is as follows:

* Write the conditional distributions $\pi(u | v)$ and $\pi(v | u)$

* Initialize $u^{(0)}$ and $v^{(0)}$ and any values in the domains of their distributions

* For $t = 1, \ldots, M$,

    - Sample $u^{(t)} \sim \pi(\cdot | v^{(t-1)})$

    - Sample $v^{(t)} \sim \pi(\cdot | u^{(t)})$

The samples are called approximate samples because (1) they are not independent and (2) the sampler must be run for a
sufficiently long period before it returns samples approximating the target distribution $\pi(u, v)$. We want iid draws
from the target distribution, which is also referred to as the stationary distribution, but the sampler does not immediately
give us these. To overcome this, we can *burn in* to the Markov chain to reach samples from the stationary distribution
(solving problem 2) and *thin* the chain (solving problem 1) to make the samples independent.

To make the last point concrete, suppose we know it takes 10,000 samples to reach the target distribution and that
$$
    \text{Corr}\big[u^{(t)}, u^{(t + s)}\big] =
    \begin{cases}
        1 &\text{if}\quad t - s = 0, \\
        \rho(t - s) &\text{if}\quad |t - s| < 4, \\
        0 &\text{if}\quad |t - s| \geqslant 4.
    \end{cases}
$$
Above, $\rho(t - s)$ is any decreasing, nonnegative function in the real numbers which is bounded above by one. 
This is a formal way of saying the correlation between variables is nonzero for any choice of samples within four timesteps
of one another but points from four or more timesteps apart have zero correlation and are effectively "independent". In this framework, suppose we want to obtain $N$ samples from the target distribution $\pi(u, v)$. This will mean drawing 
$M = 10000 + 4N$ total samples from the sampler. We will drop the first 10,000 samples as the burn-in phase, then take every
fourth sample from the remaining chain as our independent samples from the target posterior distribution.

### Guidelines for a Bayesian Statistical Model

In their book *Bayesian Data Analysis*, Gelman et al. recommend the following steps to fitting a Bayesian statistical model:

1. Setting up a full probability model for the data and parameters in the model using an appropriate likelihood function for
the data and a prior which captures any available knowledge about the underlying scientific problem.

2. Conditioning on observed data to calculate and interpret a posterior distribution.

3. Evaluating the posterior distribution using diagnostic plots to assess model fit, and conclusions that can be drawn from
the posterior distribution.

This book and other introductory literature are listed in the references at the end of this notebook.

## Example: Sampling From a T Distribution

The following example is similar to the one shown in Section 3.3 of *Bayesian Data Analysis*
([pdf here](http://www.stat.columbia.edu/~gelman/book/)). The density for a $t$ distribution with $\nu$ degrees of freedom,
location $\mu \in \mathbb{R}$, and scale $\sigma > 0$ is
\begin{equation}
    f(t | \nu, \mu, \sigma) =
    \frac{\Gamma(\frac{\nu + 1}{2})}{\sigma\sqrt{\nu\pi}\Gamma(\frac{\nu}{2})}\left[
        1 + \frac{\left(\frac{x - \mu}{\sigma}\right)^2}{\nu}
    \right]^{-\frac{\nu + 1}{2}}.
    \tag{2}
\end{equation}
It turns out that the distribution can be written hierarchically as
\begin{align*}
    X | \tau &\sim N(\mu, \tau^{-1}\sigma^2), \\
    \tau &\sim \text{Gamma}\left(\frac{\nu}{2}, \frac{\nu}{2}\right),
\end{align*}
where
$$
    f(\tau | \frac{\nu}{2}, \frac{\nu}{2}) = 
    \frac{(\nu/2)^{\nu/2}}{\Gamma(\frac{\nu}{2})} \tau^{\nu/2 - 1} e^{-\frac{\nu\tau}{2}}
$$
is the density of a gamma distribution. The density in $(2)$ is obtained by marginalizing $X | \tau$ over $\tau$:
\begin{align*}
    \int_{0}^{\infty} N(\mu, \tau^{-1}\sigma^2) \text{Gamma}\left(\frac{\nu}{2}, \frac{\nu}{2}\right) d\tau
    &= 
        \int_{0}^{\infty} \frac{\sqrt{\tau}}{\sqrt{2\pi\sigma^2}}
        e^{-\frac{\tau}{2}\left(\frac{x - \mu}{\sigma}\right)^2}
        \frac{(\frac{\nu}{2})^{\frac{\nu}{2}}}{\Gamma(\frac{\nu}{2})} 
        \tau^{\frac{\nu}{2} - 1} e^{-\frac{\nu\tau}{2}} d\tau \\
    &=
        \frac{(\frac{\nu}{2})^{\frac{\nu}{2}}}{\sigma\sqrt{2\pi}\Gamma(\frac{\nu}{2})} \int_{0}^{\infty}
        \tau^{\frac{\nu-1}{2}} e^{-\frac{\tau}{2}\left[
            \nu + \left(\frac{x - \mu}{\sigma}\right)^2
        \right]} d\tau \\
    &= I.
\end{align*}
After a change of variables, the integral is a gamma function. Let $y = (x - \mu) / \sigma$ and make the
substitution $2u = \tau(\nu + y^2)$ so $2du = (\nu + y^2)d\tau$. Then
\begin{align*}
    I
    &= 
        \frac{(\frac{\nu}{2})^{\frac{\nu}{2}}}{\sigma\sqrt{2\pi}\Gamma(\frac{\nu}{2})} \int_{0}^{\infty}
        \left(\frac{2u}{\nu + y^2}\right)^{\frac{\nu-1}{2}} e^{-u} \frac{2du}{\nu + y^2} \\
    &=
        \frac{(\frac{\nu}{2})^{\frac{\nu}{2}}}{\sigma\sqrt{2\pi}\Gamma(\frac{\nu}{2})}
        \left(\frac{2}{\nu + y^2}\right)^{\frac{\nu + 1}{2}}
        \underset{\Gamma(\frac{\nu + 1}{2})}{\underbrace{
            \int_{0}^{\infty} u^{\frac{\nu - 1}{2}} e^{-u} du
        }} \\
    &=
    \frac{(\frac{\nu}{2})^{\frac{\nu}{2}}}{\sigma\sqrt{2\pi}\Gamma(\frac{\nu}{2})}
    2^{\frac{\nu + 1}{2}}\left(\nu + y^2\right)^{-\frac{\nu + 1}{2}}
    \Gamma\left(\frac{\nu+1}{2}\right) \\
    &=
        \frac{
            (\frac{\nu}{2})^{\frac{\nu}{2} - \frac{\nu+1}{2}}\Gamma(\frac{\nu+1}{2})
        }{
            \sigma\sqrt{2\pi}\Gamma(\frac{\nu}{2})
        }\left(1 + \frac{y^2}{2}\right)^{-\frac{\nu + 1}{2}}\\
    &=
        \frac{\Gamma(\frac{\nu + 1}{2})}{\sigma\sqrt{\nu\pi}\Gamma(\frac{\nu}{2})}\left[
            1 + \frac{\left(\frac{x - \mu}{\sigma}\right)^2}{\nu}
        \right]^{-\frac{\nu + 1}{2}}.
\end{align*}
I've shown almost every algebraic step and verified they are correct. If any step does not make sense, feel free to
contact me and I will be happy to explain it. I highly recommend working through this derivation with a pen
and paper yourself. At the start, write both densities in the hierarchical model and the target density. Then
make sure each step in this derivation makes sense.